In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from datetime import datetime

import time 
#Neural Nets
import tensorflow as tf

# Add the parent directory to the Python path to allow imports from tests/
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from dotenv import load_dotenv

from mkt_data_ETL.data_load_and_transform import get_data, get_top_mkt_cap_stocks


import warnings
import random

load_dotenv()
# Suppress all warnings
warnings.filterwarnings("ignore")

In [ ]:

# =============================================================================
# REPRODUCIBILITY SETUP - Set seeds for deterministic results
# =============================================================================
def set_seeds(seed=42):
    """
    Set seeds for reproducible results across all random number generators.
    
    Args:
        seed (int): The seed value to use for all random number generators
    """
    # Set Python's built-in random seed
    random.seed(seed)
    
    # Set NumPy seed
    np.random.seed(seed)
    
    # Set TensorFlow seeds
    tf.random.set_seed(seed)
    
    # Set environment variables for deterministic behavior
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
    
    # Configure TensorFlow for deterministic operations
    tf.config.experimental.enable_op_determinism()
    
    print(f"All seeds set to: {seed}")
    print("Deterministic operations enabled for reproducible results.")

# Set the seed for reproducible results
SEED = 42
set_seeds(SEED)

def verify_seed_settings():
    """
    Verify that all seed settings are properly configured.
    This function can be called to check reproducibility setup.
    """
    print("=== Reproducibility Verification ===")
    print(f"SEED value: {SEED}")
    print(f"PYTHONHASHSEED: {os.environ.get('PYTHONHASHSEED', 'Not set')}")
    print(f"TF_DETERMINISTIC_OPS: {os.environ.get('TF_DETERMINISTIC_OPS', 'Not set')}")
    print(f"TF_CUDNN_DETERMINISTIC: {os.environ.get('TF_CUDNN_DETERMINISTIC', 'Not set')}")
    print("=====================================")

# Uncomment the line below to verify seed settings
# verify_seed_settings()

In [ ]:
# Get data 
stock_prices_df, stock_shares_amount_df, mkt_cap_df, spx_index, removed_companies = get_data()
top_100_mkt_cap_df, top_100_mkt_cap_prices_df=get_top_mkt_cap_stocks(stock_prices_df=stock_prices_df, 
stock_mkt_cap_df=mkt_cap_df)


# Calculate daily returns
stocks_returns    = np.log(top_100_mkt_cap_prices_df / top_100_mkt_cap_prices_df.shift(1))
sp500_idx_returns =  np.log(spx_index / spx_index.shift(1))

# Rolling volatility
window_size = 252*2 # 2 years
Sigma_df= stocks_returns.rolling(window=window_size).cov(pairwise=True)
Sigma_df = Sigma_df.dropna()

# Get the first date in the cleaned DataFrame
START_DATE = Sigma_df.index.get_level_values(0)[0]

# Filter dataframes to start from START_DATE
top_100_mkt_cap_df = top_100_mkt_cap_df.loc[START_DATE:]
stocks_returns     = stocks_returns.loc[START_DATE:]
stock_prices_df    = stock_prices_df.loc[START_DATE:]

assets = stocks_returns.columns
dates  = stocks_returns.index

n = len(assets)
T = len(dates)
Sigma_t = np.empty((T, n, n))

# Fill array with each rolling covariance matrix
for i, t in enumerate(dates):
    Sigma = Sigma_df.loc[t].reindex(index=assets, columns=assets).values
    Sigma_t[i] = Sigma


# Mkt Weights
mkt_portfolio_weights = top_100_mkt_cap_df.div(top_100_mkt_cap_df.sum(axis=1),axis=0)

#Retorno do portfólio de mercado
mkt_return = (mkt_portfolio_weights.shift(1) * stocks_returns).sum(axis=1)

In [3]:
def compute_relative_covariance(sigma: np.ndarray, pi: np.ndarray) -> np.ndarray:
    """
    Calcula a matriz de covariância relativa τ_ij^π(t) com base na covariância σ_ij(t)
    e no vetor de pesos do portfólio π(t).
    
    Args:
        sigma (np.ndarray): Matriz de covariância dos ativos (n x n).
        pi (np.ndarray): Vetor de pesos do portfólio (n,).

    Returns:
        np.ndarray: Matriz de covariância relativa τ^π (n x n).
    """
    # Covariância ativo i com portfólio: sigma_iπ = sigma @ pi
    sigma_i_pi = sigma @ pi        # (n,)
    sigma_pi_pi = pi.T @ sigma @ pi  # escalar

    # Matriz τ_ij^π = σ_ij - σ_iπ - σ_jπ + σ_ππ
    n = len(pi)
    tau = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            tau[i, j] = sigma[i, j] - sigma_i_pi[i] - sigma_i_pi[j] + sigma_pi_pi
    return tau

print("initializing tau_t calculation")
start_time = time.time()

tau_t = np.empty((T, n, n))
for t in range(T):
    tau_t[t] = compute_relative_covariance(Sigma_t[t], mkt_portfolio_weights.iloc[t].values)

end_time = time.time()
elapsed_seconds = end_time - start_time
print(f"Elapsed time: {elapsed_seconds:.2f} seconds")


# Copying data generated previously 
mu_t_df = mkt_portfolio_weights.copy()
R_t_df =  stocks_returns.copy()
 
mu_tf                   = tf.convert_to_tensor(mu_t_df.values, dtype=tf.float32) # Pesos de mercado
mkt_ret_tf              = tf.convert_to_tensor(mkt_return.values, dtype=tf.float32)              # Retorno do portfólio de mercado
R_t_tf                  = tf.convert_to_tensor(R_t_df.values, dtype=tf.float32)
tau_t_tf                = tf.convert_to_tensor(tau_t, dtype=tf.float32)


indices = np.arange(len(mu_tf))
train_idx, test_idx = train_test_split(indices, test_size=0.1, shuffle=False, random_state=SEED)

# Split datasets

#Treino
mu_train            = tf.gather(mu_tf, train_idx)
mkt_ret_train       = tf.gather(mkt_ret_tf, train_idx)
stock_returns_train = tf.gather(R_t_tf, train_idx)
tau_train           = tf.gather(tau_t_tf, train_idx)

# Teste
mu_test            = tf.gather(mu_tf, test_idx)
mkt_ret_test       = tf.gather(mkt_ret_tf, test_idx)
stock_returns_test = tf.gather(R_t_tf,  test_idx)
tau_test           = tf.gather(tau_t_tf, test_idx)

initializing tau_t calculation


NameError: name 'T' is not defined

In [ ]:
class PINN(tf.keras.Model):
    def __init__(self, input_dim=200):
        """
        Initializes the PINN model with an architecture optimized to learn
        smooth (C^2).

        Args:
            input_dim (int): The dimensionality of the network input (default: 200).
        """
        super(PINN, self).__init__()

        self.lambda1 = tf.Variable(initial_value=1.0, trainable=False, dtype=tf.float32)
        self.lambda2 = tf.Variable(initial_value=1.0, trainable=False, dtype=tf.float32)
        
        initializer = tf.keras.initializers.GlorotNormal(seed=SEED)
        l2_regularizer = tf.keras.regularizers.l2(0.001)

        self.hidden_layers = tf.keras.Sequential([
            tf.keras.layers.InputLayer(input_shape=(input_dim,)),
            tf.keras.layers.Dense(100, activation='swish', kernel_initializer=initializer),
            tf.keras.layers.Dense(100, activation='swish', kernel_initializer=initializer, kernel_regularizer=l2_regularizer),
            tf.keras.layers.Dense(100, activation='swish', kernel_initializer=initializer, kernel_regularizer=l2_regularizer),
            
        ])
        self.output_layer = tf.keras.layers.Dense(1, activation='softplus')

    @tf.function
    def call(self, inputs):
        """
        Executa a passagem para a frente (forward pass).
        A lógica de separação e concatenação foi removida por ser redundante.
        """
        z = self.hidden_layers(inputs)
        return self.output_layer(z)

In [ ]:
###########################################################
## Model Compilation ##
###########################################################

model =  PINN(input_dim=100)

steps_bounds = [1000]
lr_values = [1e-2, 1e-3]
lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(steps_bounds, lr_values)
optimizer = tf.keras.optimizers.Adam(lr_schedule, beta_1=0.9, beta_2=0.95)
optimizer_self_adp = tf.keras.optimizers.Adam(1e2, beta_1=0.9, beta_2=0.99)


#######################################
## Objects to store training metrics ##
#######################################

epochs = 10

grad_norms_per_epoch = {}
mkt_weights_per_epoch = {}
drift_per_epoch = {}

avg_epoch_loss_vect = []
grad_norms_batch_vect     = []
mkt_weights_per_batch_vect = []
drift_per_batch_vect = []

###########################################
## Initialize self adaptative parameters ##
###########################################

self_adp_lambda1 = tf.cast(tf.zeros([1,1]), dtype=tf.float64)
self_ada_lambda2 = tf.cast(tf.zeros([1,1]), dtype=tf.float64)

self_adp_lambda1 = tf.Variable(self_adp_lambda1)
self_ada_lambda2 = tf.Variable(self_ada_lambda2)

for epoch in range(epochs):
    poch_loss = 0
    num_batches = 0

    ################
    ## Train Step ##
    ################

    total_loss, equation_error, positivity_penalty, g_t, pi_t, gradient_self_adp = train_step(model, optimizer, mu_train, stock_returns_train, mkt_ret_train, tau_train)


    #####################################
    ## Update self-adaptive parameters ##
    #####################################

    optimizer_self_adp.apply_gradients(zip(gradient_self_adp, [self_adp_lambda1, self_ada_lambda2]))
    self_adp_lambda1.assign(tf.math.softplus(self_adp_lambda1))
    self_ada_lambda2.assign(tf.math.softplus(self_ada_lambda2))



    if epoch % 1 == 0:
        print('\n',
                'Testing error for Epoch {0}:  total_loss --> {1}, pde_loss -->{2}, bound_loss -->{3}, error --> {4}'.format(
                    epoch, total_loss.numpy(), equation_error.numpy(), positivity_penalty.numpy()))
        #print('*' * 100)



In [ ]:
def main():
    
    @tf.function
    def loss_function(mu, G_pred, grad_log_G_pred, HESS_G_pred, ret, ret_mkt, tau, adp_lambda1, ada_lambda2):
        """
        Custom loss function for the PINN.

        Args:
            mu (tf.Tensor): Market weights.
            G_pred (tf.Tensor): Value of G predicted by the network.
            grad_log_G_pred (tf.Tensor): Gradient of log of G predicted by the network.
            HESS_G_pred (tf.Tensor): Hessian of G predicted by the network.
            ret (tf.Tensor): Stock returns.
            ret_mkt (tf.Tensor): The market return.
            tau (tf.Tensor): Relative covariance matrix.

        Returns:
            _type_: Loss values
        """
        ###################################################################################
        # # Functionally Generated Portfolio - Generated by the Neural Network function # #
        ###################################################################################

        inner_prod = tf.reduce_sum(mu * grad_log_G_pred, axis=1)
        pi_t = ((grad_log_G_pred + 1) - tf.expand_dims(inner_prod, axis=1))*mu

        # Shift weights to consider trade date at the end of the day
        pi_t_shifted = tf.concat([pi_t[:1], pi_t[:-1]], axis=0)
        # Portfolio return
        port_ret = tf.reduce_sum(pi_t_shifted * ret, axis=1)

        #########################
        # # Drift Calculation # #
        #########################
        T = tf.shape(pi_t)[0]

        mu_i = tf.expand_dims(mu, axis=2)     # (T, n, 1)
        mu_j = tf.expand_dims(mu, axis=1)     # (T, 1, n)

        # internal product μ_i μ_j: shape (T, n, n)
        mu_outer = mu_i * mu_j

        # elementwise: H * μ_i * μ_j * τ
        elementwise = HESS_G_pred * mu_outer * tau  # shape (T, n, n)

        # sum over i and j (last two dimensions)
        summed = tf.reduce_sum(elementwise, axis=[1, 2])  # shape (T,)
        dg_t = -0.5 * summed / G_pred

        # #########################
        # # # Drift integration # #
        # #########################
        base = tf.range(T)


        weights = tf.ones_like(dg_t, dtype=dg_t.dtype)
        if T % 2 == 1:  
            weights = tf.where(
                (base % 2 == 1) & (base != 0) & (base != T-1),
                tf.constant(4.0, dtype=dg_t.dtype),
                weights
            )
            weights = tf.where(
                (base % 2 == 0) & (base != 0) & (base != T-1),
                tf.constant(2.0, dtype=dg_t.dtype),
                weights
            )
            weights *= (1.0 / 3.0)
        else:  # Trapezoidal rule
            weights = tf.where(
                (base == 0) | (base == T-1),
                tf.constant(0.5, dtype=dg_t.dtype),
                tf.constant(1.0, dtype=dg_t.dtype)
            )
        g_t = tf.reduce_sum(weights * dg_t)

        # #######################
        # # # Master Eq Error # #
        # #######################

        eps = 1e-6
        G0 = tf.clip_by_value(G_pred[0], clip_value_min=eps, clip_value_max=1e6)
        GT = tf.clip_by_value(G_pred[-1], clip_value_min=eps, clip_value_max=1e6)
        right_hand_side = tf.math.log(G0[0]) - tf.math.log(GT[0]) + g_t

        # Cumulative log return
        port_cumulative_return = tf.math.exp(tf.reduce_sum(port_ret))
        mkt_cumulative_return = tf.math.exp(tf.reduce_sum(ret_mkt))
        left_hand_side = tf.math.log(port_cumulative_return) - tf.math.log(mkt_cumulative_return)

        equation_error = tf.square(left_hand_side - right_hand_side)  # Smooth, differentiable
        positivity_penalty = tf.square(tf.nn.relu(-g_t)) # Penalize negative drift

        total_loss = adp_lambda1 * equation_error + ada_lambda2 * positivity_penalty

        return total_loss, equation_error, positivity_penalty, g_t, pi_t

    @tf.function
    def train_step(model, optimizer, x, ret, ret_mkt, tau):
        with tf.GradientTape(persistent=True) as tape2:
            with tf.GradientTape() as tape1:
                G_pred = model(x)
                log_G_pred = tf.math.log(G_pred + 1e-7)
                grad_log_G = tape1.gradient(log_G_pred, x)
                grad_G       = tape1.gradient(G_pred, x) 

            H_G = tape2.batch_jacobian(grad_G, x)  # Hessian of G(x)
            total_loss, equation_error, positivity_penalty, g_t, pi_t = loss_function(mu=x, G_pred= G_pred, grad_log_G_pred= grad_log_G, HESS_G_pred= H_G, ret= ret, ret_mkt= ret_mkt, tau= tau, adp_lambda1= self_adp_lambda1, ada_lambda2= self_ada_lambda2)

        gradient_nn_wt = tape1.gradient(total_loss, model.trainable_variables)
        (gradient_self_adp) = tape1.gradient(total_loss, [self_adp_lambda1, self_ada_lambda2])

        optimizer.apply_gradients(zip(gradient_nn_wt, model.trainable_variables))

        return total_loss, equation_error, positivity_penalty, g_t, pi_t, gradient_self_adp

    #######################
    ## Model Compilation ##
    #######################

    model =  PINN(input_dim=100)

    steps_bounds = [1000]
    lr_values = [1e-2, 1e-3]
    lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(steps_bounds, lr_values)
    optimizer = tf.keras.optimizers.Adam(lr_schedule, beta_1=0.9, beta_2=0.95)
    optimizer_self_adp = tf.keras.optimizers.Adam(1e2, beta_1=0.9, beta_2=0.99)


    #######################################
    ## Objects to store training metrics ##
    #######################################

    epochs = 10

    grad_norms_per_epoch = {}
    mkt_weights_per_epoch = {}
    drift_per_epoch = {}

    avg_epoch_loss_vect = []
    grad_norms_batch_vect     = []
    mkt_weights_per_batch_vect = []
    drift_per_batch_vect = []

    ###########################################
    ## Initialize self adaptative parameters ##
    ###########################################

    self_adp_lambda1 = tf.cast(tf.zeros([1,1]), dtype=tf.float64)
    self_ada_lambda2 = tf.cast(tf.zeros([1,1]), dtype=tf.float64)

    self_adp_lambda1 = tf.Variable(self_adp_lambda1)
    self_ada_lambda2 = tf.Variable(self_ada_lambda2)

    for epoch in range(epochs):
        poch_loss = 0
        num_batches = 0

        ################
        ## Train Step ##
        ################

        total_loss, equation_error, positivity_penalty, g_t, pi_t, gradient_self_adp = train_step(model, optimizer, mu_train, stock_returns_train, mkt_ret_train, tau_train)


        #####################################
        ## Update self-adaptive parameters ##
        #####################################

        optimizer_self_adp.apply_gradients(zip(gradient_self_adp, [self_adp_lambda1, self_ada_lambda2]))
        self_adp_lambda1.assign(tf.math.softplus(self_adp_lambda1))
        self_ada_lambda2.assign(tf.math.softplus(self_ada_lambda2))



        if epoch % 1 == 0:
            print('\n',
                    'Testing error for Epoch {0}:  total_loss --> {1}, pde_loss -->{2}, bound_loss -->{3}, error --> {4}'.format(
                        epoch, total_loss.numpy(), equation_error.numpy(), positivity_penalty.numpy()))

if __name__ == "__main__":
    main()